<h1>Section 1: Graph-Tool<h1>
        This section includes testing of the basic Graph-Tool functionalities

In [120]:
from graph_tool.all import *

In [2]:
g = Graph(directed=False)
v1 = g.add_vertex()
v2 = g.add_vertex()
v3 = g.add_vertex()
e1 = g.add_edge(v1, v2)
e2 = g.add_edge(v1, v3)

In [3]:
graph_draw(g, vertex_text=g.vertex_index, output="two-nodes.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1a8314a60, at 0x1a8314fd0>

In [4]:
print(v1.out_degree())
print(e1.source(), e1.target())

2
0 1


In [5]:
vlist = g.add_vertex(10)
print(len(list(vlist)))

10


In [ ]:
e3 = g.add_edge(4, 5)

In [ ]:
for v in g.iter_vertices():
    print(v)
for e in g.iter_edges():
    print(e)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
[0, 1]
[0, 2]
[4, 5]


In [ ]:
g.get_out_degrees(g.get_vertices())

array([2, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint64)

In [ ]:
from numpy.random import randint

g = Graph()
g.add_vertex(100)

# insert some random links
num_edges = 10
for s,t in zip(randint(0, 100, num_edges), randint(0, 100, num_edges)):
    g.add_edge(g.vertex(s), g.vertex(t))
    
#print edges
print("edges added:")
for i in g.iter_edges():
    print(i)

vprop_double = g.new_vertex_property("double")            # Double-precision floating point
v = g.vertex(10)
vprop_double[v] = 3.1416
print("property of vertex {}: {}".format(v, vprop_double[v]))

vprop_vint = g.new_vertex_property("vector<int>")         # Vector of ints
v = g.vertex(40)
vprop_vint[v] = [1, 3, 42, 54]
print("property of vertex {}: {}".format(v, vprop_vint[v]))

eprop_dict = g.new_edge_property("object")                # Arbitrary Python object.
e = g.edges().next()
eprop_dict[e] = {"name": "foo", "weight": 42} 
print("property of edge {}: {}".format(e, eprop_dict[e]))

gprop_bool = g.new_graph_property("bool")                 # Boolean
gprop_bool[g] = True


edges added:
[6, 7]
[15, 98]
[18, 27]
[39, 22]
[50, 26]
[54, 10]
[71, 78]
[73, 7]
[75, 84]
[79, 25]
property of vertex 10: 3.1416
property of vertex 40: array([ 1,  3, 42, 54], dtype=int32)
property of edge (6, 7): {'name': 'foo', 'weight': 42}


In [ ]:
# See what the initializations are for each of the graph properties
for prop in ["int", "vector<int>", "double", "vector<double>", "object", "bool"]:
    gprop = g.new_graph_property(prop)
    g.graph_properties["custom property"] = gprop
    print(prop, g.graph_properties["custom property"])

int 0
vector<int> array([], dtype=int32)
double 0.0
vector<double> array([], dtype=float64)
object None
bool 0


In [ ]:
g.vp.custom_property = vprop = g.new_vertex_property("bool")
g.vp.custom_property[g.vertex(0)] = True
print(g.vp.custom_property[g.vertex(0)])

1


In [129]:
import os
working_dir = '.' #placeholder

'''
Saves the graph g to {target path} with name {name}
'''
def save_graph(g, name, target_path):
    abs_path = os.path.join(working_dir, target_path)
    g.save("{}.xml.gz".format(name))
    

'''
Loads the graph g from {source path} 

Returns: the loaded graph 
'''
def load_graph(source_path):
    return load_graph(source_path)


In [145]:
to_graph(clusters, driving_forces, agents, opinions)

In [144]:
%matplotlib inline
import graph_tool.all as gt
import os 
import matplotlib.pyplot as plt

'''
converts the following to graphs and output in {target_directory}:
    
    - agent ids
    - agent intrinsics
    - driving forces
    - opinion
    
    
Input:

    agents:           numpy array of agent intrinsics
    clusters:         {"cluster_i": a list of agent ids}
    driving_forces:   numpy array of the driving forces
    opinions:         binary numpy array of the opinions
    dynamic:          use matplotlib for dynamic visualization instead of saving the graph directly
    target_directory: output directory
    
    
The graphs this function produces:

    1.Opinion
    2.Driving force
    3.Agent intrinsics (can be many separate features)
    
    
TODO: how to use matplotlib or export sequence of plots to video/gif for convinient visualization?
 
    
'''


def to_graph(clusters, driving_forces, agents, opinions, dynamic=False, target_directory=""):
    
    if dynamic:
        plt.switch_backend("cairo")
    
    # directed graph, each cluster is a clique
    g = gt.Graph() 
    num_agents = len(agents)
    num_clusters = clusters.shape[0]
    g.add_vertex(num_agents)
    cluster_assignment = np.zeros(num_agents)
    for cluster in range(num_clusters):
        agents_in_cluster = clusters[cluster, :]
        cluster_assignment[agents_in_cluster] = cluster
        for a1 in agents_in_cluster:
            for a2 in agents_in_cluster:
                if a1 != a2:
                    g.add_edge(g.vertex(a1), g.vertex(a2)) 
    
    add_edges_between_clusters(g, cluster_assignment)
    
    # add edges between clusters
    #show_heatmap(g)
            
    #vprop_attributes = g.new_vertex_property("vector<double>")
    vprop_opinion = g.new_vertex_property("int")
    for v in g.iter_vertices():
        vprop_opinion[v] = opinions[v]
    
    # draw opinions
    #layout_list = ['fruchterman_reingold_layout', 'sfdp_layout']
    state = gt.minimize_nested_blockmodel_dl(g)
    
    #pos = gt.radial_tree_layout(g) # try different layouts
    pos = gt.sfdp_layout(g)
    deg = g.degree_property_map("in")
    draw_order = np.concatenate(clusters)# by cluster
    fill_color = vprop_opinion
    #print(fill_color)
    
    #gt.draw_hierarchy(state, output="experiment.pdf", vprops = vprop_opinion)
    
    
    gt.graph_draw(g, pos=pos, 
                  vertex_fill_color=fill_color,#vorder=draw_order,
                  vertex_size = 25,
                  vertex_pen_width = 2,
                  edge_pen_width=2,
                  #edge_color='',
                  inline=True,
                  output=os.path.join(target_directory, "experiment.pdf"))
    
#     # draw driving forces

In [123]:
'''
add edges between clusters of the graph g

currently addds random edges between clusters

for every agent, add edge to another agent that is no in the same cluster with some probability
'''
def add_edges_between_clusters(g, cluster_assignment):
    p = 0.05
    num_agents = len(cluster_assignment)
    for a in range(num_agents):
        for a_ in range(num_agents):
            if not (a == a_ or cluster_assignment[a] == cluster_assignment[a_]):
                if p > np.random.rand(1):
                    g.add_edge(g.vertex(a), g.vertex(a_))
 

In [124]:
def show_heatmap(g):
    state = gt.minimize_blockmodel_dl(g)
    b = gt.contiguous_map(state.get_blocks())
    state = state.copy(b=b)
    e = state.get_matrix()
    B = state.get_nonempty_B()
    plt.matshow(e.todense()[:B, :B])
    plt.show()

<h2>Section 2: the model will be implemented here<h2>

In [125]:
import numpy as np
import scipy
from scipy.stats import norm
import math

In [126]:
'''
Global configurations
'''
k_alpha = 1
k_beta = .5
E_profit = 10
K_self_coupling = -2

num_clusters = 5
num_agents = 20

In [127]:
from random import randint
import random


'''
Create {num_clusters} random clusters
if cluster_specific = True, use initialise_driving_force() to initialise cluster-specific driving force

Returns: 
           cluster_elements:  {"cluster_i": numpy array of agent ids in cluster i}
           driving_forces:    numpy array of driving forces (properly-/0-initialised)
           agents:            numpy array of agent intrinsics
           opinions:          a numpy binary array of the opinions

'''

def create_clusters(num_clusters, num_agents, cluster_specific=False):
    assert num_agents >= num_clusters >= 1
  
        
    #initialise driving forces
    if cluster_specific:
        driving_forces = initialise_driving_forces(num_agents)
    else:
        driving_forces = np.zeros(num_agents)
        
    #initialise agents intrinsics
    agents = initialise_agents(num_agents)
    
    #initialise clusters
    cluster_elements = initialise_clusters(num_agents, num_clusters)
    cluster_elements = np.array(cluster_elements)
    print("elems:", cluster_elements)

    #initialise opinions
    opinions = initialise_opinions(num_agents)
        
    return [cluster_elements, driving_forces, agents, opinions]
        
    
'''
Initialises the clusters
You can define initialization schemes here
currently does uniform partition

Retuns: a numpy array containing the driving force of each agent

'''   

def initialise_clusters(num_agents, n):
    l = list(range(num_agents))
    random.shuffle(l)
    res = np.array([l[i::n] for i in range(n)])
    return res
'''
Initialises the driving forces for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the driving force of each agent

'''   
def initialise_driving_forces(num_agents):
    driving_forces = np.random.rand(num_agents) #hard-coded for now
    return driving_forces
    
'''
Initialises the intrinsics for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the intrinsics of each agent
'''
def initialise_agents(num_agents):
    return np.zeros(num_agents) #hard-coded for now


'''
Initialises the opinions for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the opinions of each agent
'''
def initialise_opinions(num_agents):
    return np.random.randint(2, size=num_agents) #hard-coded for now




In [128]:
clusters, driving_forces, agents, opinions = create_clusters(num_clusters, num_agents)
print("clusters: {}\n\n driving forces: {}\n\n agents:{}\n\n opinions:{}".format(clusters, driving_forces, agents, opinions))

elems: [[ 8 17  6  5]
 [ 1  2  9 18]
 [14 12 19 11]
 [ 3 16  7  0]
 [15  4 10 13]]
clusters: [[ 8 17  6  5]
 [ 1  2  9 18]
 [14 12 19 11]
 [ 3 16  7  0]
 [15  4 10 13]]

 driving forces: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

 agents:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

 opinions:[0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0]


In [134]:
'''
    updates the agent intrinsics (optional), opinions and driving forces 
'''

# TODO: need to fix this
def update_driving_forces(clusters, driving_forces, agents, opinions, update_agents=False):
    # update each cluster locally resp. each agent
    print("updating clusters ...")
    updated_driving_force = np.zeros(len(driving_forces))
    for n in range(len(clusters)):
        print("cluster ", n)
        #agents_ids = clusters["cluster_{}".format(n)]
        clusters = np.array(clusters)
        print("clusters:", clusters)
        agents_ids = clusters[n, :]
        cluster_size = len(agents_ids)

        # number of different opinions 
        num_diffops = np.zeros(len(opinions))
        for a in agents_ids:
            num_diffops[a] = np.sum([x for x in opinions[agents_ids] if x != a])
            
        # update driving forces    
        for a in agents_ids:
            #driving_forces
            updated_driving_force[a] = E_profit + num_diffops[a] * k_alpha
            print(updated_driving_force)
            print("agent {} obtained a driving force of {}".format(a, updated_driving_force[a]))
        print("resulting global df: {}\n".format(updated_driving_force))
        
            
    return updated_driving_force #currently it returns the unchanged agent intrinsics

In [ ]:
upd_agts, upd_df = update_driving_forces(driving_forces, opinions, clusters, opinions)
upd_agts, upd_df

updating clusters ...
cluster  0
agent 11 obtained a df of 10.0
agent 2 obtained a df of 10.0
agent 9 obtained a df of 10.0
agent 12 obtained a df of 10.0
resulting global df: [ 0.  0. 10.  0.  0.  0.  0.  0.  0. 10.  0. 10. 10.  0.  0.  0.  0.  0.
  0.  0.]

cluster  1
resulting global df: [ 0.  0. 10.  0.  0.  0.  0.  0.  0. 10.  0. 10. 10.  0.  0.  0.  0.  0.
  0.  0.]

cluster  2
agent 7 obtained a df of 10.0
agent 5 obtained a df of 10.0
agent 0 obtained a df of 10.0
agent 6 obtained a df of 10.0
agent 13 obtained a df of 10.0
agent 15 obtained a df of 10.0
agent 18 obtained a df of 10.0
resulting global df: [10.  0. 10.  0.  0. 10. 10. 10.  0. 10.  0. 10. 10. 10.  0. 10.  0.  0.
 10.  0.]

cluster  3
agent 4 obtained a df of 10.0
agent 3 obtained a df of 10.0
agent 10 obtained a df of 10.0
resulting global df: [10.  0. 10. 10. 10. 10. 10. 10.  0. 10. 10. 10. 10. 10.  0. 10.  0.  0.
 10.  0.]

cluster  4
agent 17 obtained a df of 10.0
agent 14 obtained a df of 10.0
resulting globa

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([10.,  0., 10., 10., 10., 10., 10., 10.,  0., 10., 10., 10., 10.,
        10., 10., 10.,  0., 10., 10.,  0.]))

In [106]:
'''
Given the current state, returns a new state

Input: 
       -current_state: (agents' instrinsics, opinions, probabilities of change, driving forces)
       -f:             the state transition function
       
       
Returns:
       The new state f(current_state)
'''
# TODO: adapt this 
# You can choose whether to unpack current_state or not
def update_state(current_state):
    return transition_function(current_state)


'''
You can define state transition functions here
'''
def transition_function(state):
       agents = state[0]
       hi = hi_calculation(state[3])
        #print(state)
       state[2] = change_probs(state[3])

       for i in range(len(agents)): 
              new_opinion = opinion(state[1][i], state[2][i])
              state[1][i] = new_opinion
       return state


In [126]:
'''
Part of modelling the transition function

'''
def normalise_driving_force(driving_forces, hi):
    return driving_forces/hi


'''
compute the probability of change given a driving force    
'''
def change_probs(driving_forces):
    hi = np.sum(driving_forces) # the maximum possible driving force 
    lo = None
    driving_forces = normalise_driving_force(driving_forces, hi )
    probs = [math.erf(driving_forces[i]) if driving_forces[i] < hi else math.erf(hi) for i in range(len(driving_forces)) ]
    return probs


'''
The new opinion {x} of an agent given the prob. of change {p}

Can serve as a state transition function
'''
def opinion(x, p):
    res = (np.random.rand(1))[0]
    return 1-x if res < p else x


<h1>Section 3: Simulation<h1>
    1.Preprocessing (e.g. what graph models to use)<br>
    2.Simulate the model and visualise the result (e.g. for each timestep)

In [11]:
import csv



cluster_max_index = [117, 167, 211, 246]

cluster_names = ["Peoria","Bloomington" , "Quincy" ,"Galesburg"]



def get_edges_array(cluster_max):

    peoria_cluster_edges = []

    bloomington_cluster_edges = []

    quincy_cluster_edges = []

    galesburg_cluster_edges = []

    with open('medical_innovationver3.csv',newline='', encoding='utf-8') as f:

        reader = csv.reader(f)

        row_num = 0

        for row in reader:
            if row_num == 0:
                row_num+=1
            else:
                
                args = row[0].split(";")

                node1 = int(args[0])

                

                node2 = int(args[1][1:-1])

                fr_ad_dis = [int(args[2][1:-1]), int(args[3][1:-1]), int(args[5][1:-1])]

                id = int(args[4][1:-1])

                edge_properties= [node1, node2, fr_ad_dis[0], fr_ad_dis[1], fr_ad_dis[2]]

                if (node1 <= cluster_max[0] and node2 <= cluster_max[0]):

                    

                    peoria_cluster_edges.append(edge_properties)

                elif(node1 <= cluster_max[1] and node2 <= cluster_max[1]): 

                    bloomington_cluster_edges.append(edge_properties)

                elif(node1 <= cluster_max[2] and node2 <= cluster_max[2]): 

                    quincy_cluster_edges.append(edge_properties)

                else: 

                    galesburg_cluster_edges.append(edge_properties)

    return peoria_cluster_edges, bloomington_cluster_edges, quincy_cluster_edges, galesburg_cluster_edges    



###

# edges arrays arguments: 

#   - node1 (from)

#   - node2 (to) 

#   - friendship relation 

#   - advice relation

#   - discussion relation

#  ###

peoria_edges, bloomington_edges, quincy_edges, galesburg_edges = get_edges_array(cluster_max_index)


In [43]:
# Simulation configs
num_steps = 42
visualise_per = 5

# Simulation loop, e.g.
#
# model = preprocess(data)
#
# clusters, driving_forces, agents, opinions = create_clusters(num_clusters, num_agents)
# current_state = (clusters, driving_forces, agents, opinions)
#
# for i in range(num_steps):
#    if i % visualise_per = 0:
#       visualise(current_state)
#    current_state = update_state(current_state, transition_function)
#
# graph = to_graph(current_state)
# save_graph(graph, graph_name, save_path)
#
#
#
# Note that in the current version we need to convert the current state to a graph
# first in order to visualise it -> maybe improve this



In [108]:
# --test--  idea : visualize clusters individually
    
import matplotlib.pyplot as plt

def visualise(current_state):
  plt.figure(figsize=(20,10),dpi=80)
  axes=plt.subplot(111)

  cluster = current_state[0]
  opinions = current_state[3]
  cluster = np.array(cluster)
    
  for i in range(len(cluster)):
    agent_ids = cluster[i, :]
    
    np.random.seed(2000)
    y = np.random.standard_normal((1000, 2))
    plt.figure(figsize=(7,5))
    
    # assign different colors to different opinion
    ops = opinions[agent_ids] #np arrays
    colours = ['r' if o == 0 else 'b' for o in ops]
    x_pos = np.linspace(0, 1, len(agent_ids))
    y_pos = np.random.rand(len(x_pos))
    
    plt.scatter(x_pos, y_pos, c=colours, s=40, cmap=plt.cm.Spectral)
    plt.grid(True)
    plt.xlabel('1st')
    plt.ylabel('2nd')
    plt.title('cluster_{}'.format(i))
    plt.show()

# visualise(current_state)


In [136]:
'''
[cluster_elements, driving_forces, agents, opinions]
'''
def main():
    elems = create_clusters(num_clusters, num_agents, cluster_specific=True)
    probs = change_probs(elems[1])
    
    cluster_elements, driving_forces, agents, opinions = elems[0], elems[1], elems[2], elems[3]   
    # [cluster_elements, driving_forces, agents, opinions]
    
    # cs: [cluster_elements, driving_forces, agents, opinions]
    #  current_state: (agents' instrinsics, opinions, probabilities of change, driving forces)
    current_state = [agents, opinions, probs, driving_forces]
    for x in range(5):
        print("iteration", x)
        #visualise([init[0], current_state[3], current_state[0], current_state[1]])
        print(driving_forces)
        current_state[3] = update_driving_forces(clusters, driving_forces, agents, opinions, update_agents=False)
        current_state = update_state(current_state)

        

main()

elems: [[ 3 12 10  0]
 [15 11 16  4]
 [14 19 17  8]
 [ 2  6  5  1]
 [18  9 13  7]]
iteration 0
[0.19781142 0.77772861 0.69797421 0.08465719 0.58558062 0.4945531
 0.04351066 0.47439866 0.90765958 0.61117943 0.19068572 0.31587508
 0.07017903 0.51441192 0.0215703  0.59218831 0.19867751 0.71608139
 0.52121245 0.11175903]
updating clusters ...
cluster  0
clusters: [[ 2 10  4 15]
 [ 8 16  1 11]
 [ 9 18  3 13]
 [17 12 19 14]
 [ 0  5  7  6]]
[ 0.  0. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
agent 2 obtained a driving force of 13.0
[ 0.  0. 13.  0.  0.  0.  0.  0.  0.  0. 13.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
agent 10 obtained a driving force of 13.0
[ 0.  0. 13.  0. 13.  0.  0.  0.  0.  0. 13.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
agent 4 obtained a driving force of 13.0
[ 0.  0. 13.  0. 13.  0.  0.  0.  0.  0. 13.  0.  0.  0.  0. 13.  0.  0.
  0.  0.]
agent 15 obtained a driving force of 13.0
resulting global df: [ 0.  0. 13.  0. 13.  0.  0.  0.  0.  0. 13